# Clustering and PCA

### Mushroom Dataset

Podeis obtener el conjunto de datos en el siguiente enlace:

[Mushroom Dataset](https://www.kaggle.com/uciml/mushroom-classification)

Como podréis comprobar, hay muchas variables, todas ellas categóricas, por lo que exploraciones con scatterplot no nos serán útiles como en otros casos.

La variable a predecir ``class`` es binaria.


In [154]:
# Carga de librerías, las que hemos considerado básicas, añadid lo que queráis :)

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

### Leer conjunto de datos y primer vistazo

In [155]:
df = pd.read_csv("data/agaricus-lepiota.data")

df.head(5)

,p,x,s,n,t,p.1,f,c,n.1,k,...,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u
0,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [156]:
# imprimimos los nombres de las columnas
df.columns

Index(['p', 'x', 's', 'n', 't', 'p.1', 'f', 'c', 'n.1', 'k', 'e', 'e.1', 's.1',
       's.2', 'w', 'w.1', 'p.2', 'w.2', 'o', 'p.3', 'k.1', 's.3', 'u'],
      dtype='object')

In [157]:
# imprimos el número de valores únicos por columna
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valores únicos")


p: 2 valores únicos
x: 6 valores únicos
s: 4 valores únicos
n: 10 valores únicos
t: 2 valores únicos
p.1: 9 valores únicos
f: 2 valores únicos
c: 2 valores únicos
n.1: 2 valores únicos
k: 12 valores únicos
e: 2 valores únicos
e.1: 5 valores únicos
s.1: 4 valores únicos
s.2: 4 valores únicos
w: 9 valores únicos
w.1: 9 valores únicos
p.2: 1 valores únicos
w.2: 4 valores únicos
o: 3 valores únicos
p.3: 5 valores únicos
k.1: 9 valores únicos
s.3: 6 valores únicos
u: 7 valores únicos


In [158]:
# renombramos las colunmas para que sean más descriptivas
df.columns = [
    'clase',                   # class
    'forma_sombrero',          # cap-shape
    'superficie_sombrero',     # cap-surface
    'color_sombrero',          # cap-color
    'magulladuras',            # bruises
    'olor',                    # odor
    'union_laminas',           # gill-attachment
    'espaciado_laminas',       # gill-spacing
    'tamano_laminas',          # gill-size
    'color_laminas',           # gill-color
    'forma_tallo',             # stalk-shape
    'raiz_tallo',              # stalk-root
    'superficie_tallo_arriba_anillo',  # stalk-surface-above-ring
    'superficie_tallo_abajo_anillo',   # stalk-surface-below-ring
    'color_tallo_arriba_anillo',       # stalk-color-above-ring
    'color_tallo_abajo_anillo',        # stalk-color-below-ring
    'tipo_velo',              # veil-type
    'color_velo',             # veil-color
    'numero_anillos',         # ring-number
    'tipo_anillo',            # ring-type
    'color_esporas',          # spore-print-color
    'poblacion',              # population
    'habitat',                # habitat
]


| Nombre de columna                    | Valores posibles (código = significado)                                                                 |
|-------------------------------------|----------------------------------------------------------------------------------------------------------|
| clase                      | e=comestible, p=venenoso                      |
| forma_sombrero                      | b = campana, c = cónica, x = convexa, f = plana, k = con protuberancia, s = hundida                      |
| superficie_sombrero                 | f = fibrosa, g = con ranuras, y = escamosa, s = lisa                                                     |
| color_sombrero                      | n = marrón, b = beige, c = canela, g = gris, r = verde, p = rosado, u = púrpura, e = rojo, w = blanco, y = amarillo |
| magulladuras                        | t = sí, f = no                                                                                           |
| olor                                | a = almendra, l = anís, c = creosota, y = a pescado, f = fétido, m = mohoso, n = ninguno, p = picante, s = especiado |
| union_laminas                       | a = adherida, d = descendente, f = libre, n = entallada                                                  |
| espaciado_laminas                   | c = cerrado, w = apiñado, d = separado                                                                   |
| tamano_laminas                      | b = ancho, n = estrecho                                                                                   |
| color_laminas                       | k = negro, n = marrón, b = beige, h = chocolate, g = gris, r = verde, o = naranja, p = rosado, u = púrpura, e = rojo, w = blanco, y = amarillo |
| forma_tallo                         | e = ensanchado, t = afilado                                                                              |
| raiz_tallo                          | b = bulboso, c = maza, e = igual, r = enraizado, ? = faltante                   |
| superficie_tallo_arriba_anillo     | f = fibrosa, y = escamosa, k = sedosa, s = lisa                                                          |
| superficie_tallo_abajo_anillo      | f = fibrosa, y = escamosa, k = sedosa, s = lisa                                                          |
| color_tallo_arriba_anillo          | n = marrón, b = beige, c = canela, g = gris, o = naranja, p = rosado, e = rojo, w = blanco, y = amarillo |
| color_tallo_abajo_anillo           | n = marrón, b = beige, c = canela, g = gris, o = naranja, p = rosado, e = rojo, w = blanco, y = amarillo |
| tipo_velo                           | p = parcial, u = universal                                                                               |
| color_velo                          | n = marrón, o = naranja, w = blanco, y = amarillo                                                        |
| numero_anillos                      | n = ninguno, o = uno, t = dos                                                                            |
| tipo_anillo                         | c = telaraña, e = efímero, f = ensanchado, l = grande, n = ninguno, p = colgante, s = envainado, z = zona |
| color_esporas                       | k = negro, n = marrón, b = beige, h = chocolate, r = verde, o = naranja, u = púrpura, w = blanco, y = amarillo |
| poblacion                           | a = abundante, c = agrupada, n = numerosa, s = dispersa, v = varias, y = solitaria                      |
| habitat                             | g = pasto, l = hojas, m = praderas, p = senderos, u = urbano, w = desechos, d = bosque                   |


In [159]:
# imprimimos el número de valores faltantes por columna
df.isnull().sum()

clase                             0
forma_sombrero                    0
superficie_sombrero               0
color_sombrero                    0
magulladuras                      0
olor                              0
union_laminas                     0
espaciado_laminas                 0
tamano_laminas                    0
color_laminas                     0
forma_tallo                       0
raiz_tallo                        0
superficie_tallo_arriba_anillo    0
superficie_tallo_abajo_anillo     0
color_tallo_arriba_anillo         0
color_tallo_abajo_anillo          0
tipo_velo                         0
color_velo                        0
numero_anillos                    0
tipo_anillo                       0
color_esporas                     0
poblacion                         0
habitat                           0
dtype: int64

Podemos ver a simple vista que no hay valores faltantes, pero mirando el "diccionario" de variables hay una columna llamada "s.1" que ahora se llama "raiz_tallo" tiene una variable llamada "?" que corresponde a valor faltante. Procedemos a imprimir la cantidad de valores "?" hay en el dataset, más necesario en la columna "raiz_tallo".

In [160]:
df.isin(['?']).sum()

clase                                0
forma_sombrero                       0
superficie_sombrero                  0
color_sombrero                       0
magulladuras                         0
olor                                 0
union_laminas                        0
espaciado_laminas                    0
tamano_laminas                       0
color_laminas                        0
forma_tallo                          0
raiz_tallo                        2480
superficie_tallo_arriba_anillo       0
superficie_tallo_abajo_anillo        0
color_tallo_arriba_anillo            0
color_tallo_abajo_anillo             0
tipo_velo                            0
color_velo                           0
numero_anillos                       0
tipo_anillo                          0
color_esporas                        0
poblacion                            0
habitat                              0
dtype: int64

In [161]:
df['raiz_tallo'].value_counts()

raiz_tallo
b    3776
?    2480
e    1119
c     556
r     192
Name: count, dtype: int64

Confirmamos que hay 2480 valores faltantes en la columna "raiz_tallo" y los convertimos a "nan".

In [162]:
df['raiz_tallo'] = df['raiz_tallo'].replace('?', np.nan)

In [163]:
df["raiz_tallo"].value_counts(dropna=False)

raiz_tallo
b      3776
NaN    2480
e      1119
c       556
r       192
Name: count, dtype: int64

### Exploración de datos

In [164]:
# Descripción del conjunto de datos, estándard.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8123 entries, 0 to 8122
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   clase                           8123 non-null   object
 1   forma_sombrero                  8123 non-null   object
 2   superficie_sombrero             8123 non-null   object
 3   color_sombrero                  8123 non-null   object
 4   magulladuras                    8123 non-null   object
 5   olor                            8123 non-null   object
 6   union_laminas                   8123 non-null   object
 7   espaciado_laminas               8123 non-null   object
 8   tamano_laminas                  8123 non-null   object
 9   color_laminas                   8123 non-null   object
 10  forma_tallo                     8123 non-null   object
 11  raiz_tallo                      5643 non-null   object
 12  superficie_tallo_arriba_anillo  8123 non-null   

In [165]:
# Información sobre el tipo de datos de cada feature.
print("Todos los datos son de tipo 'object', es decir son categóricos.")

Todos los datos son de tipo 'object', es decir son categóricos.


#### Calcular el número de nulos de cada feature

In [166]:
# Igual que otras veces, una linea, contar los nulos por variable.
print("El dataset no contiene valores nulos en ninguna de sus columnas, pero en la columna 'raiz_tallo' hay 248 valores que son '?' \nque representan datos perdidos y los hemos reemplazado por NaN.")

El dataset no contiene valores nulos en ninguna de sus columnas, pero en la columna 'raiz_tallo' hay 248 valores que son '?' 
que representan datos perdidos y los hemos reemplazado por NaN.


#### Buscar valores extraños. Para ello, ver los valores únicos en cada feature

In [167]:
# Obtener un nuevo dataframe de dos columnas donde en la primera estén las features (features) y en la otra los valores únicos
# asociados (n_values).

features = []
valores_unicos = []

for col in df.columns:
    features.append(col)
    valores_unicos.append(list(df[col].unique()))

resumen = pd.DataFrame({
    'feature': features,
    'n_values': valores_unicos
})

print(resumen)

resumen.to_csv("data/resumen_features.csv", index=False)
print("El nuevo dataframe ha sido guardado en 'resumen_features.csv'.")



                           feature                              n_values
0                            clase                                [e, p]
1                   forma_sombrero                    [x, b, s, f, k, c]
2              superficie_sombrero                          [s, y, f, g]
3                   color_sombrero        [y, w, g, n, e, p, b, u, c, r]
4                     magulladuras                                [t, f]
5                             olor           [a, l, p, n, f, c, y, s, m]
6                    union_laminas                                [f, a]
7                espaciado_laminas                                [c, w]
8                   tamano_laminas                                [b, n]
9                    color_laminas  [k, n, g, p, w, h, u, e, b, r, y, o]
10                     forma_tallo                                [e, t]
11                      raiz_tallo                     [c, e, b, r, nan]
12  superficie_tallo_arriba_anillo                 

#### Tratar aquellos valores que entendamos que sean nulos


In [168]:
# Imputaciones. Podéis quitar esos puntos (fila entera), imputar con la moda o dejar ese valor como una posibilidad más.

#### Mirad cuántos valores hay en cada feature, ¿Todas las features aportan información? Si alguna no aporta información, eliminadla

In [169]:
# Dejar por el camino si procede.

#### Separar entre variables predictoras y variables a predecir

In [170]:
# La variable que trata de predecir este conjunto de datos es 'class'.
y =
X =

SyntaxError: invalid syntax (3592957914.py, line 2)

#### Codificar correctamente las variables categóricas a numéricas

In [ ]:
# One Hot Encoder (una linea).

#### Train test split

In [ ]:
# Os lo dejamos a todos igual
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## PCA

Es un conjunto de datos del que aún no hemos visto nada (no tenemos graficas) así que vamos a hacer algunas. Tenemos el problema de que son muchas variables, **PCA al rescate**: le pedimos que nos de dos dimensiones y las pintamos, sabemos que serán **aquellas que retengan más información**.

In [ ]:
pca =       # metodo de sklearn
pca.fit(X_train)

# Representar en un scatterplot y poner en color las etiquetas de entrenamiento

Parece que está bastante separadito, parece que a ojo mucho se puede ver :)

Igualmente, vamos a entrenar un clasificador a ver qué tal lo hace antes de editar más

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Definir el clasificador y el número de estimadores
# 2. Entrenar en train
# 3. Calcular la precisión sobre test

Es un conjunto sencillo y Random Forest es muy bueno en su trabajo, Igualmente, vamos a ver qué tamaño tenemos de dataset:


In [ ]:
X_train.shape

¿Muchas features no? Vamos a reducir las usando PCA.

In [ ]:
n_features = # definir un rango de valores a probar
scores = []

for n in n_features:

    # Hacer PCA sobre X_train
    # 1. Definir PCA
    # 2. Aprender PCA sobre X_train

    # Entrenar Random Forest
    # 1. Definir el RF
    # 2. Entrenar clasificador

    # Guardar el score


sns.lineplot(x=n_features, y=scores)


Vale, estamos viendo que a partir de unas 10 features ya tenemos el score que queríamos y además hemos reducido las variables a un 10% de las que teníamos, incluso menos que las variables originales.

## Clustering

Viendo que el conjunto de datos es sencillito, podemos intentar hacer algo de clustering a ver qué información podemos obtener.

El primer paso va a ser importar la función de Kmeans de sklearn, y a partir de ahi, vamos a buscar el valor óptimo de clusters. Como hemos visto anteriormente, este valor lo obtenemos, por ejemplo, del codo de la gráfica que representa el total de las distancias de los puntos a los centros de los clusters asociados. Os dejo la página de la documentación de sklearn para que lo busquéis:

[K-Means on sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

Con esto solo hay que ahora generar los modelos de kmeans, evaluar y pintar la gráfica para los valores de ``k`` que establezcais.




In [ ]:
from sklearn.cluster import KMeans

scores = []
k_values = # definir un rango
for a in k_values:

    # Definir Kmeans y ajustar
    # Guardar la predicción

sns.lineplot(x=k_values, y=scores)

Con el valor que hayáis obtenido de la gráfica, podéis obtener una buena aproximación de Kmeans y con ello podemos pasar a explorar cómo de bien han separado la información los distintos clusters. Para ello, se va a hacer un ``catplot``, seaborn os lo hará solito. Con esto lo que se pretende ver es la distribución de la varaible a predecir en función del cluster que haya determinado Kmeans.

In [ ]:
# Aprender Kmeans con el valor de K obtenido.

kmeans = # Definir y entrenar Kmeans.

# Preparar el catplot.


# Pintar.
ax = sns.catplot(col=, x=, data=, kind='count',col_wrap=4)

Vamos a ver qué tal queda esto pintado. Para ello, repetimos el scatterplot de antes pero usando como color el cluster asignado por kmeans.

In [ ]:
# Entrenar PCA para representar.

# Usar un color por cada cluster.


¿Es bastante parecido no? No es tan bueno como el Random Forest, pero ha conseguido identificar bastante bien los distintos puntos del dataset sin utilizar las etiquetas. De hecho, el diagrama de factor que hemos visto antes muestra que solo un par de clusters son imprecisos. Si no hubieramos tenido etiquetas esta aproximacion nos hubiera ayudado mucho a clasificar los distintos tipos de hongos.